In [7]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from AdjustedRandomForest import train

In [8]:
'''
# CONTROL - NO MISLABELLING
scores_my = []
scores_std = []

for _ in range(10):
    X, y = make_classification(n_samples=3000, n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=2,  random_state = 1, flip_y=0.0000001)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1)
    
    rf, corrected_y = train(RandomForestClassifier, X_train, y_train, np.unique(y), 5)
    y_test_pred = rf.predict(X_test)
    scores_my.append(metrics.accuracy_score(y_test, y_test_pred))

    rf = RandomForestClassifier(n_estimators=5, criterion='entropy')
    rf.fit(X_train, y_train)
    y_test_pred = rf.predict(X_test)
    scores_std.append(metrics.accuracy_score(y_test, y_test_pred))

print(f"Test\nIndividual Accuracies: {scores_my}\nAverage Accuracy: {np.mean(scores_my)}\n")
print(f"Control\nIndividual Accuracies: {scores_std}\nAverage Accuracy: {np.mean(scores_std)}")
'''

'\n# CONTROL - NO MISLABELLING\nscores_my = []\nscores_std = []\n\nfor _ in range(10):\n    X, y = make_classification(n_samples=3000, n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=2,  random_state = 1, flip_y=0.0000001)\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1)\n    \n    rf, corrected_y = train(RandomForestClassifier, X_train, y_train, np.unique(y), 5)\n    y_test_pred = rf.predict(X_test)\n    scores_my.append(metrics.accuracy_score(y_test, y_test_pred))\n\n    rf = RandomForestClassifier(n_estimators=5, criterion=\'entropy\')\n    rf.fit(X_train, y_train)\n    y_test_pred = rf.predict(X_test)\n    scores_std.append(metrics.accuracy_score(y_test, y_test_pred))\n\nprint(f"Test\nIndividual Accuracies: {scores_my}\nAverage Accuracy: {np.mean(scores_my)}\n")\nprint(f"Control\nIndividual Accuracies: {scores_std}\nAverage Accuracy: {np.mean(scores_std)}")\n'

In [9]:
# MISLABELLING
scores_my = []
scores_std = []
N_CLASSES = 2
MISLABELLING =0.4

for _ in range(10):
    X, y = make_classification(n_samples=3000, n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=N_CLASSES, flip_y=0.00000001)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    y_mislabelled = np.copy(y_train)

    for i in range(int(len(y_mislabelled) * MISLABELLING)):
        y_mislabelled[i] = (y_mislabelled[i] + random.randint(1, N_CLASSES - 1)) % N_CLASSES

    print(sum(i==j for i, j in zip(y_mislabelled, y_train)))
    np.random.shuffle(y_mislabelled)

    rf, corrected_y = train(RandomForestClassifier, X_train, y_mislabelled, np.unique(y), 5)
    y_test_pred = rf.predict(X_test)
    print(sum(i==j for i, j in zip(y_test_pred, y_train)))
    scores_my.append(metrics.accuracy_score(y_test, y_test_pred))

    rf = RandomForestClassifier(n_estimators=5, criterion='entropy')
    rf.fit(X_train, y_mislabelled)
    y_test_pred = rf.predict(X_test)
    scores_std.append(metrics.accuracy_score(y_test, y_test_pred))

print(f"Test\nIndividual Accuracies: {scores_my}\nAverage Accuracy: {np.mean(scores_my)}\n")
print(f"Control\nIndividual Accuracies: {scores_std}\nAverage Accuracy: {np.mean(scores_std)}")

1206
(4020, 2)
(196, 2)
(168, 2)
(856, 2)
(766, 2)
501
1206
(4020, 2)
(80, 2)
(62, 2)
(22, 2)
(52, 2)
495
1206
(4020, 2)
(390, 2)
(292, 2)
(722, 2)
(700, 2)
521
1206
(4020, 2)
(78, 2)
(340, 2)
(342, 2)
(342, 2)
478
1206
(4020, 2)
(436, 2)
(622, 2)
(516, 2)
(606, 2)
480
1206
(4020, 2)
(68, 2)
(22, 2)
(22, 2)
(130, 2)
497
1206
(4020, 2)
(78, 2)
(166, 2)
(166, 2)
(184, 2)
490
1206
(4020, 2)
(38, 2)
(30, 2)
(52, 2)
(44, 2)
505
1206
(4020, 2)
(528, 2)
(206, 2)
(1576, 2)
(1356, 2)
490
1206
(4020, 2)
(20, 2)
(16, 2)
(930, 2)
(926, 2)
490
Test
Individual Accuracies: [0.5141414141414141, 0.5070707070707071, 0.05454545454545454, 0.5, 0.5060606060606061, 0.5040404040404041, 0.5141414141414141, 0.49696969696969695, 0.5898989898989899, 0.5181818181818182]
Average Accuracy: 0.47050505050505054

Control
Individual Accuracies: [0.5434343434343434, 0.5070707070707071, 0.4696969696969697, 0.5393939393939394, 0.48383838383838385, 0.5262626262626262, 0.503030303030303, 0.509090909090909, 0.522222222222222

In [12]:
from collections import defaultdict

e = [[0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.01980198, 0.98019802] ,
 [0.0474934 , 0.9525066 ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.01980198, 0.98019802] ,
 [0.01980198, 0.98019802] ,
 [0.40530303, 0.59469697] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.0474934 , 0.9525066 ] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.0474934 , 0.9525066 ] ,
 [0.40530303, 0.59469697] ,
 [0.40530303, 0.59469697] ,
 [0.9125    , 0.0875    ] ,
 [0.9125    , 0.0875    ] ,
 [0.40530303, 0.59469697] ,
 [0.40530303, 0.59469697] ,
 [0.01980198, 0.98019802] ,
 [0.0474934 , 0.9525066 ]]

y = [1,1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0 ,0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1 ,0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1]
K = 1
# Dictionary to store lists of values corresponding to each unique label
grouped_values = defaultdict(list)

# Group values by their corresponding label
for label, probability in zip(y, e):
    grouped_values[label].append(probability[label])

print(grouped_values)

# Compute mean for each label
means = {label: np.mean(probabilities) for label, probabilities in grouped_values.items()}
standard_deviations = {label: np.std(probabilities) for label, probabilities in grouped_values.items()}

print(means)
print(standard_deviations)

selection = []

for i in range(len(X)):
    if e[i][y[i]] >= (means[y[i]] + K * standard_deviations[y[i]]) or np.isclose(e[i][y[i]], means[y[i]] + K * standard_deviations[y[i]], atol=1e-8):
        selection.append(i)

defaultdict(<class 'list'>, {1: [0.59469697, 0.9525066, 0.9525066, 0.98019802, 0.9525066, 0.9525066, 0.59469697, 0.9525066, 0.9525066, 0.9525066, 0.59469697, 0.9525066, 0.9525066, 0.9525066, 0.9525066, 0.59469697, 0.59469697, 0.9525066, 0.59469697, 0.9525066, 0.59469697, 0.9525066, 0.9525066, 0.9525066, 0.98019802, 0.98019802, 0.9525066, 0.9525066, 0.9525066, 0.9525066, 0.59469697, 0.59469697, 0.59469697, 0.98019802, 0.9525066], 0: [0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.9125, 0.40530303, 0.9125, 0.9125, 0.40530303, 0.40530303, 0.40530303, 0.9125, 0.40530303, 0.40530303, 0.9125, 0.9125]})
{1: 0.8534400108571428, 0: 0.8364204545}
{1: 0.16386823576274817, 0: 0.18110554302143972}


IndexError: list index out of range